In [19]:
import polars as pl

In [20]:
# Group inspections per year, remove duplicates, and merge IDs from both sheets.
early_years = (
    pl.read_csv(
        "./data/inspections_16-18.csv",
        dtypes=[pl.Utf8, pl.Int64, pl.Int64, pl.Int64, pl.Int64],
    )
    .drop(["Grand Total"])
    .with_columns(pl.all().fill_null(0))
)
later_years = pl.read_csv(
    "./data/inspections_19-23.csv",
    dtypes=[
        pl.Utf8,
        pl.Int64,
        pl.Int64,
        pl.Int64,
        pl.Int64,
        pl.Int64,
        pl.Int64,
    ],
).drop(["Grand Total"])
merged_df = early_years.join(
    later_years, left_on="Unit_Code", right_on="Unit", how="outer"
)
grouped_df = merged_df.groupby("Unit_Code").agg(
    pl.col("2016").sum(),
    pl.col("2017").sum(),
    pl.col("2018").sum(),
    pl.col("2019").sum(),
    pl.col("2020").sum(),
    pl.col("2021").sum(),
    pl.col("2022").sum(),
    pl.col("2023").sum(),
)
grouped_df = grouped_df.with_columns(
    grouped_df.select(pl.all().exclude("Unit_Code"))
    .sum(axis=1)
    .alias("Sum Inspections")
)

In [27]:
# Import and munge payments data.
payments_df = (
    pl.read_csv(
        "./data/payments-15-23.csv",
        dtypes=[
            pl.Utf8,
            pl.Int64,
            pl.Int64,
            pl.Int64,
            pl.Int64,
            pl.Int64,
            pl.Int64,
            pl.Int64,
            pl.Int64,
            pl.Int64,
        ],
    )
    .drop(["Grand Total"])
    .filter(pl.col("Unit_ID") != "Grand Total")
    .with_columns(pl.all().fill_null(0))
)
payments_df = payments_df.with_columns(
    payments_df.select(pl.all().exclude("Unit_ID"))
    .sum(axis=1)
    .alias("Sum Payments")
)

Unit_ID,2015,2016,2017,2018,2019,2020,2021,2022,2023,Sum Payments
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""1001324""",0,0,0,0,0,0,7154,11379,1896,20429
"""1002618""",10434,20517,19479,20598,24574,14452,18744,12496,6248,147542
"""1003440""",3870,7509,6816,2840,0,0,0,0,0,21035
"""1004080""",0,10103,9946,10848,11489,12156,11548,1133,0,67223
"""1005039""",4620,7959,8808,8808,11712,13401,12676,14796,3411,86191
"""1006397""",0,0,11886,0,16005,15744,15372,14240,7120,80367
"""1006553""",5667,11292,11493,12984,14750,14839,17184,11312,5656,105177
"""1006784""",5430,10860,5430,4818,7620,4525,0,0,0,38683
"""1006998""",7472,11934,10260,10818,11349,13738,19950,7980,0,93501


In [34]:
# Combine
inspection_payment_df = grouped_df.join(
    payments_df, left_on="Unit_Code", right_on="Unit_ID", how="outer"
).with_columns(pl.all().fill_null(0))
inspection_payment_df = inspection_payment_df.select(
    ["Unit_Code", "Sum Inspections", "Sum Payments"]
)
inspection_payment_df

Unit_Code,Sum Inspections,Sum Payments
str,i64,i64
"""4112043""",1,0
"""7035047""",2,0
"""4000444""",15,70389
"""5000336""",10,80211
"""1057120""",11,78856
"""4002111""",13,55745
"""1007604""",15,71934
"""4018094""",1,0
"""4231260""",4,0
